# Export

Jurassic exports code cells tagged with `//| export` comment into corresponding
`ts` modules. Code cells with Deno tests are grouped into test module (This is
likely to change in the future when
[this](https://github.com/denoland/deno/issues/27434) gets fixed).

Let's use a few examples to illustrate how exports work assuming we are looking
at the cells within `foo.ipynb`

```ts
//| export

export const bar = () => "hey";
```

converts to `foo.ts` with

```ts
export const bar = () => "hey";
```

If you need to share code between different cells, make sure to use `//| export`
in both:

```ts
//| export
const foo = () => "foo";
```

```ts
//| export

export const bar = () => foo();
```

Notice how both cells are tagges with `//| export` to make sure both pieces of
code make it to the final `foo.ts` module.

Keep in mind, `//| export` tag does not automatically export actual ts
function/class/type, it merely moves it to the target ts module. You are
responsible for managing visibility of the code using `export` TS keyword.

In [ ]:
//| export

import path from "node:path";
import { isDirective, loadNb } from "jurassic/notebooks.ts";
import {
  getNotebooksToProcess,
  removeDuplicateImports,
} from "jurassic/utils.ts";
import type { Config } from "jurassic/config.ts";

In [ ]:
//| export

const moduleHeader = (moduleName: string): string =>
  `// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: ${moduleName}\n\n`;

const processNb = async (
  nbPath: string,
  moduleName: string,
): Promise<string[]> => {
  const nb = await loadNb(nbPath);
  // we only need exportable cells
  const exportCells = nb.cells.filter((cell) => cell.isExportable);
  const testCells = nb.cells.filter((cell) => cell.isTestCell);
  return [
    removeDuplicateImports(
      exportCells.reduce(
        // get rid of directives, we want code only
        (acc, cell) =>
          acc + cell.source.filter((s) => !isDirective(s)).join(""),
        moduleHeader(moduleName),
      ),
    ),
    testCells.length !== 0
      ? removeDuplicateImports(
        testCells.reduce(
          (acc, cell) => acc + cell.source.join(""),
          moduleHeader(moduleName),
        ),
      )
      : "",
  ];
};

In [ ]:
const [m, tests] = await processNb(
  path.resolve("./export.ipynb"),
  "export.ipynb",
);

await Deno.jupyter.display(
  {
    "text/markdown": "```ts\n\n" + m + "\n```",
  },
  { raw: true },
);
await Deno.jupyter.display(
  {
    "text/markdown": "```ts\n\n" + tests + "\n```",
  },
  { raw: true },
);

```ts

import path  from 'node:path';
import { isDirective, loadNb } from 'jurassic/notebooks.ts';
import { getNotebooksToProcess, removeDuplicateImports } from 'jurassic/utils.ts';
import type { Config } from 'jurassic/config.ts';
import { getConfig } from 'jurassic/config.ts';
// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: export.ipynb






const moduleHeader = (moduleName: string): string =>
  `// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: ${moduleName}\n\n`;

const processNb = async (
  nbPath: string,
  moduleName: string,
): Promise<string[]> => {
  const nb = await loadNb(nbPath);
  // we only need exportable cells
  const exportCells = nb.cells.filter((cell) => cell.isExportable);
  const testCells = nb.cells.filter((cell) => cell.isTestCell);
  return [
    removeDuplicateImports(
      exportCells.reduce(
        // get rid of directives, we want code only
        (acc, cell) =>
          acc + cell.source.filter((s) => !isDirective(s)).join(""),
        moduleHeader(moduleName),
      ),
    ),
    testCells.length !== 0
      ? removeDuplicateImports(
        testCells.reduce(
          (acc, cell) => acc + cell.source.join(""),
          moduleHeader(moduleName),
        ),
      )
      : "",
  ];
};



export const exportNb = async (
  notebookPath?: string | undefined,
  config?: Config | undefined,
): Promise<void> => {
  notebookPath = notebookPath || ".";
  config = config || await getConfig();
  const notebooksToProcess: string[] = getNotebooksToProcess(
    notebookPath,
    config.nbsPath,
  );

  try {
    await Deno.stat(config.outputPath);
    await Deno.remove(config.outputPath, { recursive: true });
  } catch {
    // noop
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".ts");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.outputPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });

    const [moduleCode, testCode] = await processNb(
      path.resolve(config.nbsPath, notebook),
      notebook,
    );

    await Deno.writeTextFile(
      path.join(config.outputPath, outputFile),
      moduleCode,
    );
    if (testCode) {
      await Deno.writeTextFile(
        path.join(config.outputPath, outputFile.replace(".ts", ".test.ts")),
        [moduleCode, "/** ----------------tests ---------------- **/", testCode]
          .join("\n\n"),
      );
    }
  }
};
```

```ts

import { assert } from 'jsr:@std/assert';
import { getTestConfig } from 'jurassic/config.ts';
import { getProjectRoot } from 'jurassic/utils.ts';
// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: export.ipynb





Deno.test("export", async (t) => {
  // set things up, let's recreate mini project structure inside a temp dir
  const td = await Deno.makeTempDir({});

  // recreate nbs dire in temp dir and copy notebooks there
  await Deno.mkdir(`${td}/nbs`);

  Deno.copyFileSync(
    path.resolve(getProjectRoot(), "nbs/export.ipynb"),
    `${td}/nbs/export.ipynb`,
  );

  await t.step("test export", async () => {
    await exportNb("./", getTestConfig(td));

    // make sure output modules are created
    const exportContent = await Deno.readTextFile(`${td}/jurassic/export.ts`);

    // spot check content inside the output modules
    assert(exportContent.includes("export const exportNb"));

    // pretty print temp directory structure
    // await Deno.jupyter.display({
    //   "text/markdown": "```md\n" + dirListing(td) + "\n```",
    // }, { raw: true });
  });
});
```

In [ ]:
// | export

import path from "node:path";
import { getConfig } from "jurassic/config.ts";

export const exportNb = async (
  notebookPath?: string | undefined,
  config?: Config | undefined,
): Promise<void> => {
  notebookPath = notebookPath || ".";
  config = config || await getConfig();
  const notebooksToProcess: string[] = getNotebooksToProcess(
    notebookPath,
    config.nbsPath,
  );

  // nuke the output directory ONLY if we are processing multiple files
  // exportNb can be called inline at the bottom of the notebook to re-export on update
  // we want to keep the rest of the codebase intact in this case
  if (notebooksToProcess.length > 1) {
    try {
        await Deno.stat(config.outputPath);
        await Deno.remove(config.outputPath, { recursive: true });
    } catch {
        // noop
    }
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".ts");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.outputPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });

    const [moduleCode, testCode] = await processNb(
      path.resolve(config.nbsPath, notebook),
      notebook,
    );

    await Deno.writeTextFile(
      path.join(config.outputPath, outputFile),
      moduleCode,
    );
    if (testCode) {
      await Deno.writeTextFile(
        path.join(config.outputPath, outputFile.replace(".ts", ".test.ts")),
        [moduleCode, "/** ----------------tests ---------------- **/", testCode]
          .join("\n\n"),
      );
    }
  }
};

In [ ]:
import { assert } from "jsr:@std/assert";
import { getTestConfig } from "jurassic/config.ts";
import { getProjectRoot } from "jurassic/utils.ts";

Deno.test("export", async (t) => {
  // set things up, let's recreate mini project structure inside a temp dir
  const td = await Deno.makeTempDir({});

  // recreate nbs dire in temp dir and copy notebooks there
  await Deno.mkdir(`${td}/nbs`);

  Deno.copyFileSync(
    path.resolve(getProjectRoot(), "nbs/export.ipynb"),
    `${td}/nbs/export.ipynb`,
  );

  await t.step("test export", async () => {
    await exportNb("./", getTestConfig(td));

    // make sure output modules are created
    const exportContent = await Deno.readTextFile(`${td}/jurassic/export.ts`);

    // spot check content inside the output modules
    assert(exportContent.includes("export const exportNb"));

    // pretty print temp directory structure
    // await Deno.jupyter.display({
    //   "text/markdown": "```md\n" + dirListing(td) + "\n```",
    // }, { raw: true });
  });
});